In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "../models/Mistral-7B-v0.1/models--mistralai--Mistral-7B-v0.1/snapshots/7231864981174d9bee8c7687c24c8344414eae6b"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="cuda",
    torch_dtype=torch.float16,
    local_files_only=True
)


# Quick hook test on layer 16
activations = {}
def hook16(m, inp, out): activations["l16"] = out.detach().cpu()
target_layer = model.model.layers[16].post_attention_layernorm
target_layer.register_forward_hook( hook16)

inputs = tokenizer("What is the mechanism of aspirin?", return_tensors="pt").to("cuda")
with torch.no_grad(): _ = model(**inputs)
print("L16 shape:", activations["l16"].shape)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

L16 shape: torch.Size([1, 9, 4096])
